In [1]:
import pennylane as qml 
from pennylane import numpy as np

# import all the functions from the modules
from lpenny.dvr_hamiltonian_gen_I import * 
from lpenny.dvr_hamiltonian_gen_II import *
from lpenny.dvr_vqe import *
from lpenny.greedy_search_algorithm import * 
from lpenny.pot_gen_1d import *
from lpenny.pot_gen_2d import *

In [4]:
from braket.devices import Devices
device_arn = Devices._OQC.Lucy
s3 = ("erdabravest-2005", "dvr-vqe")
mol_params = cr2_params
params = [2.8, 4.0]
N = 5
spin = 1

dvr_options = { 'type': '1d',
        'box_lims': (params[0], params[1]),
        'dx': (params[1] - params[0]) / (2**N),
        'count':(2**N)
        } 

h_dvr =  gen_ham(mol_params, spin, dvr_options)
h_dvr_p = qml.pauli_decompose(h_dvr)
params = [7.54029847,  2.32973712, 4.16465528, 2.93240341, -3.18132556,  6.39754618, 4.92988485,  2.97029896, -4.64164573,  1.11399732, -1.17733492]


# Define the quantum circuit with the parameters  
@hybrid_job(device=device_arn)
def circuit(params): 
    for i in range(5):
        qml.SX(wires=i)
        qml.RZ(params[i], wires=i)
    qml.SX(wires=0)
    qml.ECR(wires=[2,1])
    qml.SX(wires=2)
    qml.SX(wires=3)
    qml.SX(wires=4)
    qml.SX(wires=1)
    qml.RZ(params[5], wires=2)
    qml.RZ(params[6], wires=4)
    qml.RZ(params[7], wires=1)
    qml.ECR(wires=[3, 2])
    qml.RZ(params[10], wires=0)
    qml.ECR(wires=[1, 0])
    qml.SX(wires=2)
    qml.RZ(params[8], wires=3)
    qml.SX(wires=1)
    qml.RZ(params[9], wires=2)
    qml.SX(wires=3)
    qml.ECR(wires=[4, 3])
    return qml.expval(h_dvr_p)

[ 2.6  2.8  3.   3.1  3.2  3.3  3.4  3.5  3.6  3.8  4.   4.4  4.8  5.2
  5.6  6.   6.4  6.8  7.2  7.6  8.   9.  10.  11.  12.  13.  14.  15.
 16.  17.  18.  19.  20.  22.  24.  28.  32.  36.  40. ]


QuantumFunctionError: Invalid device. Device must be a valid PennyLane device.